In [8]:
import ee
import geemap
import time
import requests
import json
import os
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd
import topojson as tp
#ee.Authenticate()
#geemap.ee_search()
ee.Initialize()
Map = geemap.Map()

In [9]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:
t1=2023
# filter Landsat 8 SR imagery 
prefire_start = str(t1-1)+'-09-01'
prefire_end = str(t1-1)+'-11-30'

# Now set the same parameters for AFTER the fire.
postfire_start = str(t1-1)+'-12-01'
#postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
postfire_end = str(t1)+'-07-31'

def addNBR(image):
    nbr = image.normalizedDifference(['B5', 'B7']).rename("nbr")
    image = image.addBands(nbr)
    return image
area =  ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')
dem = ee.Image('USGS/SRTMGL1_003')
dem_mask=dem.gt(2000)
fires_firms = ee.ImageCollection('FIRMS')\
        .select('T21')\
        .filterDate(prefire_start,postfire_end)\
        .filterBounds(area)

fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
fires_firmsrc = fires_firmsmx.gte(50)
fires_firmsvc= fires_firmsrc.reduceToVectors(**{
'geometry':area,
'crs': 'EPSG:32718',
'scale':500,
'geometryType':'polygon',
'eightConnected': False,
'bestEffort':False,
'maxPixels':9e10,
}
)

fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
    .filterMetadata('label', 'greater_than', 0)\
    .filterMetadata('count', 'greater_than', 10)
fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(500))
fires_firmsvc = fires_firmsvc.union()

prefireImCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                .filterDate(prefire_start, prefire_end) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .filterBounds(area) \
                .map(addNBR)
                
postfireImCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                .filterDate(postfire_start, postfire_end) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .filterBounds(area) \
                .map(addNBR)

prefire = prefireImCol.mosaic().clip(fires_firmsvc)
postfire = postfireImCol.mosaic().clip(fires_firmsvc)
preNBR = prefire.select('nbr')
postNBR = postfire.select('nbr')
dNBR_withoutsmooth = preNBR.subtract(postNBR)

dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=ee.Kernel.gaussian(radius=20),
)

dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))


In [11]:
viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}
Map.addLayer(dNBR_reclass, viz_dNBR, 'dNBR Clase')

EEException: Image.select: Pattern 'nbr' did not match any bands.

In [ ]:
Map.layers